# Predicción de precio de casas

In [40]:
#importar librerias
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor

In [5]:
# importar datos
melbourne_data = pd.read_csv("melb_data.csv")

In [6]:
# descripción de los datos
melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [7]:
# Primeros 5 filas.
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## Interpretación de la descripción de los datos.

Los resultados muestran 8 números para cada columna en su conjunto de datos original. La primera columna (count), muestra cuántas filas tienen valores que no faltan. Los valores faltantes surgen por muchas razones. Por ejemplo, el tamaño de una segunda habitación no se recopiló al inspeccionar una casa de una habitación. El segundo valor es la media. Debajo de eso, la desviación estándar, que mide qué tan distribuidos numéricamente están los valores. Para interpretar los valores mínimos, y máximos el 25%, 50% y 75%.  Imagine ordenar cada columna de menor a mayor valor. El primer valor (el más pequeño) es el mínimo. Si recorre un cuarto de la lista, encontrará un número que es mayor que el 25% de los valores y menor que el 75% de los valores. Ese es el valor del 25% (pronunciado "percentil 25"). Los percentiles 50 y 75 se definen de forma análoga, y el máximo es el número más grande.

In [8]:
# ¿Cuál es el tamaño de lote promedio (redondeado al número entero más cercano)?
avg_lot_size = round(melbourne_data['BuildingArea'].mean())
print(avg_lot_size)

152


In [9]:
# A partir de hoy, ¿cuántos años tiene la casa más nueva (año actual: la fecha en que se construyó)?
newest_home_age = abs(melbourne_data['YearBuilt'].max()-2021)
print(int(newest_home_age))

3


## Construyendo el modelo

Los pasos para construir y usar un modelo son: 
1. Definir: ¿Qué tipo de modelo será? ¿Un árbol de decisiones? ¿Algún otro tipo de modelo? También se especifican algunos otros parámetros del tipo de modelo. 
2. Ajustar: Captura de patrones a partir de los datos proporcionados. 
3. Predecir.
4. Evaluar: Determinar qué tan precisas son las predicciones del modelo.

### Paso 1: Especifique el objetivo de predicción

In [10]:
# imprimir la lista de columnas en el conjunto de datos para encontrar el nombre del objetivo de predicción
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [11]:
# Selección del objeto de predicción.
y = melbourne_data.Price

### Paso 2: crea X

In [12]:
# Seleccionando caracteristas
melbourne_features = ['Rooms', 'Bathroom', 'Landsize','Lattitude', 'Longtitude']

# Crear DataFrame con las caracteristicas selecciondas
X = melbourne_data[melbourne_features]

#### Revisar datos

In [13]:
# imprimir descripción o estadísticas de X
print(X.describe())

              Rooms      Bathroom       Landsize     Lattitude    Longtitude
count  13580.000000  13580.000000   13580.000000  13580.000000  13580.000000
mean       2.937997      1.534242     558.416127    -37.809203    144.995216
std        0.955748      0.691712    3990.669241      0.079260      0.103916
min        1.000000      0.000000       0.000000    -38.182550    144.431810
25%        2.000000      1.000000     177.000000    -37.856822    144.929600
50%        3.000000      1.000000     440.000000    -37.802355    145.000100
75%        3.000000      2.000000     651.000000    -37.756400    145.058305
max       10.000000      8.000000  433014.000000    -37.408530    145.526350


In [14]:
# imprime las primeras líneas 
print(X.head ())

   Rooms  Bathroom  Landsize  Lattitude  Longtitude
0      2       1.0     202.0   -37.7996    144.9984
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
3      3       2.0      94.0   -37.7969    144.9969
4      4       1.0     120.0   -37.8072    144.9941


### Paso 3: especificar y ajustar el modelo

Muchos modelos de aprendizaje automático permiten cierta aleatoriedad en el entrenamiento de modelos. Especificar un número para random_state asegura que obtendrá los mismos resultados en cada ejecución. 
Ahora tenemos un modelo ajustado que podemos usar para hacer predicciones. 
En la práctica, querrá hacer predicciones para las casas nuevas que saldrán al mercado en lugar de las casas para las que ya tenemos precios. Pero haremos predicciones para las primeras filas de los datos de entrenamiento para ver cómo funciona la función de predicción.

In [15]:
# Definir modelo. Especifique un número para random_state para garantizar los mismos resultados en cada ejecución.
melbourne_model = DecisionTreeRegressor(random_state=1)

# Modelo de ajuste
melbourne_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

### Paso 4: hacer predicciones

In [16]:
print("Haciendo predicción para las siguientes 4 casas: ")
print(X.head())
print('----------------------------------------------------')
print("Las predicciones son: ")
print(melbourne_model.predict(X.head()))

Haciendo predicción para las siguientes 4 casas: 
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
0      2       1.0     202.0   -37.7996    144.9984
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
3      3       2.0      94.0   -37.7969    144.9969
4      4       1.0     120.0   -37.8072    144.9941
----------------------------------------------------
Las predicciones son: 
[1480000. 1035000. 1465000.  850000. 1600000.]


## Validación del modelo

### Paso 1: divide tus datos

In [17]:
# Dividir los datos en datos de entrenamiento y validación, tanto para las características como para el objetivo 
# La división se basa en un generador de números aleatorios. Proporcionar un valor numérico al argumento 
# random_state garantiza que obtendremos la misma división cada vez que ejecutamos este script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0) 

### Paso 2: especifique y ajuste el modelo

In [19]:
# Definir modelo 
melbourne_model = DecisionTreeRegressor() 

# Ajustar modelo 
melbourne_model.fit(train_X, train_y) 

DecisionTreeRegressor()

### Paso 3: hacer predicciones con datos de validación

In [20]:
# obtener precios pronosticados en datos de validación 
val_predictions = melbourne_model.predict(val_X)

#### Inspeccione sus predicciones y valores reales a partir de los datos de validación.

In [22]:
# imprima las primeras predicciones de validación
print(val_predictions)

# imprima los primeros precios reales a partir de los datos de validación
print(val_y)

[1325000.  950000.  571000. ... 1575000.  819000. 1285000.]
8505     2165000.0
5523      815000.0
12852     610000.0
4818     1245000.0
12812    1160000.0
           ...    
12961     875000.0
2572     1286000.0
9503      885000.0
356       487500.0
2034     1194500.0
Name: Price, Length: 3395, dtype: float64


### Paso 4: Calcule el error absoluto medio en los datos de validación

Error absoluto medio (también llamado MAE). Analicemos esta métrica comenzando con la última palabra, error. El error de predicción para cada casa es:
$$error = real-predicho$$

In [26]:
# calculamos el error absoluto medio:
val_mae = mean_absolute_error(val_y, val_predictions)
print(val_mae)

248304.01011291114


El error que tenemos es muy alto. magínese que, en el gran mercado inmobiliario, el color de la puerta no está relacionado con el precio de la vivienda. Sin embargo, en la muestra de datos que usó para construir el modelo, todas las casas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que predigan los precios de las viviendas, por lo que verá este patrón y siempre predecirá precios altos para las casas con puertas verdes. Dado que este patrón se derivó de los datos de entrenamiento, el modelo parecerá preciso en los datos de entrenamiento. Pero si este patrón no se cumple cuando el modelo ve nuevos datos, el modelo sería muy inexacto cuando se usa en la práctica.

## Sobreajuste y  subajuste (Underfitting y Overfitting)

In [30]:
# Podemos usar una función de utilidad para ayudar a comparar las puntuaciones MAE de diferentes valores para max_leaf_nodes:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

### Paso 1: Compare diferentes tamaños de árboles

In [33]:
candidate_max_leaf_nodes = [5, 50, 500, 5000]

In [38]:
# Escriba un bucle para encontrar el tamaño de árbol ideal de candidato_max_leaf_nodes
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  354662
Max leaf nodes: 50  		 Mean Absolute Error:  266447
Max leaf nodes: 500  		 Mean Absolute Error:  231301
Max leaf nodes: 5000  		 Mean Absolute Error:  249163


In [37]:
# Almacene el mejor valor de max_leaf_nodes
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
print("\n",scores)
best_tree_size = min(scores, key=scores.get)


 {5: 354662.43726564094, 50: 266447.02204624057, 500: 231301.17567588817, 5000: 249163.62023057215}


### Paso 2: Ajustar el modelo usando todos los datos

In [44]:
# Ajuste el modelo con best_tree_size. Complete el argumento para obtener el tamaño óptimo
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=0)

# ajustar el modelo final
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=0)

## Random Forest

### Usar Random Forest

In [49]:
# Defina el modelo. Establecer random_state en 1
forest_model = RandomForestRegressor(random_state=1)

# Ajusta el modelo
forest_model.fit(train_X, train_y)

# Calcule el error absoluto medio del modelo en los datos de validación
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

180860.37877504269


#### Concluimos que un mejor modelo para nuestros datos pasa por el modelo Random Forest